## Running Qiskit Aqua algorithms using measurement error mitigation

### Introduction
To execute quantum circuits as part of aqua algorithms, we define a QuantumInstance which holds the specification for the respective backend. If we want to use measurement error mitigation for the results of the quantum circuits runs, we can put a measurement error fitting instance into the QuantumInstance. Thus, the backend defined in the QuantumInstance may be viewed as a measurement error mitigated backend.

This notebook illustrates how to run an aqua algorithm on the example of searching for the ground state of a Hamiltonian with VQE.

First, we need to import the libraries that provide backends as well as the classes that are needed to run the algorithm.

In [9]:
# Import qiskit functions and libraries
from qiskit import Aer, IBMQ
from qiskit.circuit.library import TwoLocal
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQE
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua.operators import X, Y, Z, I, CX, T, H, S, PrimitiveOp
from qiskit.providers.aer import noise

# Import error mitigation functions
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter

Then, we initialize the instances that are needed to run the algorithm.

In [11]:
# Initialize Hamiltonian
h_op =  (-1.0523732 * I^I) + \
                (0.39793742 * I^Z) + \
                (-0.3979374 * Z^I) + \
                (-0.0112801 * Z^Z) + \
                (0.18093119 * X^X)
# Initialize trial state
var_form = TwoLocal(h_op.num_qubits, ['ry', 'rz'], 'cz', reps=3, entanglement='full')
# Initialize optimizer
optimizer = SPSA(max_trials=350)
# Initialize algorithm to find the ground state
algo = VQE(h_op, var_form, optimizer)

{'optimal_parameters': {Parameter(θ[0]): 1.4832076622587647, Parameter(θ[10]): 0.07689605463521147, Parameter(θ[11]): 5.621026786702798, Parameter(θ[12]): 4.021013563435343, Parameter(θ[13]): -0.17035214656488987, Parameter(θ[14]): -1.4599685340478865, Parameter(θ[15]): -1.053304045681262, Parameter(θ[1]): 3.244819749090304, Parameter(θ[2]): -0.15015112461112395, Parameter(θ[3]): 0.9165165522860496, Parameter(θ[4]): -2.493383514648361, Parameter(θ[5]): 6.365703788455983, Parameter(θ[6]): 3.0534738114570352, Parameter(θ[7]): 1.949883177373837, Parameter(θ[8]): -3.18698897672495, Parameter(θ[9]): -2.988592058530541}, 'optimal_point': array([ 1.48320766,  0.07689605,  5.62102679,  4.02101356, -0.17035215,
       -1.45996853, -1.05330405,  3.24481975, -0.15015112,  0.91651655,
       -2.49338351,  6.36570379,  3.05347381,  1.94988318, -3.18698898,
       -2.98859206]), 'optimal_value': -1.8425421488802847, 'optimizer_time': 70.82771277427673, 'eigenvalue': (-1.8425421488802847+0j), 'eigens

Here, we define our backend which for reasons of illustration - is given by an Aer simulated noise model.
The use of a real quantum backend provided IBMQ is given in the commented part of the code.

In [12]:
# Generate a noise model
noise_model = noise.NoiseModel()
for qi in range(qubit_op.num_qubits):
    read_err = noise.errors.readout_error.ReadoutError([[0.9, 0.1],[0.25,0.75]])
    noise_model.add_readout_error(read_err, [qi])    
# Initialize the backend configuration using measurement error mitigation with a QuantumInstance 
qi_noise_model_qasm = QuantumInstance(backend=Aer.get_backend('qasm_simulator'), noise_model=noise_model, shots=8000,
                                      measurement_error_mitigation_cls=CompleteMeasFitter,
                                      measurement_error_mitigation_shots=8000)
# Intialize your TOKEN and provider with 
# provider = IBMQ.get_provider(...)
# qi_noise_model_ibmq = QuantumInstance(backend=provider = provider.get_backend(backend_name)), shots=8000,
#                                       measurement_error_mitigation_cls=CompleteMeasFitter, measurement_error_mitigation_shots=8000)

Finally, we can run the algorithm with a measurement error mitigated, shot-based simulated backend that uses a noise model.

In [14]:
# Run the algorithm
result = algo.run(qi_noise_model_qasm)
print(result)

QiskitError: 'Keyboard interrupt in parallel_map.'